In [2]:
from pyspark import SparkConf, SparkContext


sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [8]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    print(item)
    prev_v, prev_p, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_p + [next_v])

def complete(item):
    v, old_p, new_p = item[0], item[1][0], item[1][1]
    return (v, old_p if old_p is not None else new_p)

n = 4 #00 number of partitions
forward_edges = sc.textFile("/data/twitter/twitter_sample_small.txt") \
    .map(parse_edge) \
    .map(lambda x: (x[1], x[0])) \
    .partitionBy(n) \
    .persist()

start_v, end_v = 12 , 34
paths = sc.parallelize([(start_v, [start_v])]).partitionBy(n)

while True:
    candidates = paths.join(forward_edges, n).map(step).reduceByKey(min)
    paths = paths.fullOuterJoin(candidates, n).map(complete, True).persist()
    final_path = paths.lookup(end_v)
    if final_path:
        print ','.join(map(str,final_path[0]))
        break

12,422,53,52,107,20,23,274,34
